In [8]:
import pandas as pd, numpy as np
import random
from pulp import *
from ortoolpy import addbinvars

In [ ]:
# Collect response
# Parameters
address_response = "https://docs.google.com/spreadsheets/d/1ALSs4x-Z7ePV9PeFetAEJN9GgBLCweKiVCO07a2-CZc/edit?gid=2095924349#gid=2095924349"
name_sheet = "FormResponses1"
l_member = ['近藤 伸介', '市橋 香代', '藤川 慎也', '池亀 天平', '越山 太輔', '熊倉 陽介', '星野 瑞生', '森田 進', '水谷 真志', '清田 正紘', '鈴木 魁士', '市川 貴一']
l_duty = ['12/27 当直', '12/28 日当直', '12/29 日当直', '12/30 日当直', '12/31 日直', '12/31 当直', '1/1 日直', '1/1 当直', '1/2 日当直', '1/3 日当直', '1/4 日当直', '1/5 日当直']
l_rank = ['第1希望', '第2希望', '第3希望', '第4希望', '第5希望', '第6希望', '第7希望', '第8希望', '第9希望', '第10希望', '第11希望', '第12希望']

# Read G sheet
sheet_id = address_response.split('/')[5]
d_preference_src = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={name_sheet}")

# Convert
# col: member, index: rank of preference, value: index of duty
# index: duty, col: member, value: rank of preference
d_preference = pd.DataFrame(index = [i for i in range(len(l_duty))], columns = [i for i in range(len(l_member))])
for i_member, name_member in enumerate(l_member):
    if name_member in d_preference_src['お名前（敬称略）'].tolist():
        s_preference_member = d_preference_src.loc[d_preference_src['お名前（敬称略）'] == name_member, [col.startswith('日当直希望') for col in d_preference_src.columns]]
        s_preference_member = s_preference_member.iloc[len(s_preference_member)-1, :]
        for i_duty, name_duty in enumerate(l_duty):
            rank_src = s_preference_member[[name_duty in i_pref for i_pref in s_preference_member.index.tolist()]].tolist()[0]
            for i_rank, name_rank in enumerate(l_rank):
                if rank_src == name_rank:
                    d_preference.loc[i_duty, i_member] = i_rank

# Convert rank exponentially
d_preference_conv = d_preference.copy()
for i in reversed(range(len(l_duty))):
    rank_pre = i
    rank_post = i * (i + 1)/2
    d_preference_conv = d_preference_conv.replace(rank_pre, rank_post)

In [10]:
# Initialize model to be optimized
prob_assign = LpProblem()

# Binary assignment variables to be optimized, index: duty, column: member
dv_assign = pd.DataFrame(np.array(addbinvars(len(l_duty), len(l_member))))

# One assignment per member
for i_member in range(len(l_member)):
    prob_assign += (lpSum(dv_assign.loc[:, i_member]) == 1)

# One assignment per duty
for i_duty in range(len(l_duty)):
    prob_assign += (lpSum(dv_assign.loc[i_duty, :]) == 1)

# Limit to rank
'''
limit_rank = 5
for i_duty in range(len(l_duty)):
    for i_member in range(len(l_member)):
        prob_assign += (lpDot(dv_assign.loc[i_duty, i_member], d_preference.loc[i_duty, i_member]) <= limit_rank)
'''

# Variable to be minimized (sum of rank)
v_sum_rank = lpSum(lpDot(dv_assign.to_numpy(), d_preference_conv.to_numpy()))
prob_assign += v_sum_rank

# Solve
prob_assign.solve()
v_objective = value(prob_assign.objective)
print('Solved: ' + str(LpStatus[prob_assign.status]) + ', ' + str(v_objective))

# Convert to normal dataframe
d_assign = pd.DataFrame(np.vectorize(value)(dv_assign)).astype(bool)

d_match = pd.DataFrame({'duty': l_duty, 'member': None, 'rank': None})

for i_member, name_member in enumerate(l_member):
    i_duty = d_assign.loc[d_assign[i_member] == True, :].index.tolist()[0]
    rank = d_preference.loc[i_duty, i_member]
    d_match['member'].iloc[i_duty] = name_member
    d_match['rank'].iloc[i_duty] = rank

d_match

Solved: Optimal, 49.0


,duty,member,rank
0,12/27 当直,市橋 香代,0
1,12/28 日当直,池亀 天平,0
2,12/29 日当直,市川 貴一,0
3,12/30 日当直,熊倉 陽介,1
4,12/31 日直,森田 進,0
5,12/31 当直,越山 太輔,5
6,1/1 日直,星野 瑞生,3
7,1/1 当直,水谷 真志,3
8,1/2 日当直,清田 正紘,4
9,1/3 日当直,藤川 慎也,4
